# Lexy client

**WARNING**: The tests in this notebook run against the development API and database. To run isolated tests, use the following shell command instead.


```bash
make run-tests
```

In [ ]:
from lexy_py import LexyClient

lx = LexyClient()

In [ ]:
import inspect
import pytest

# Collections

### List collections

In [ ]:
lx.list_collections()

### Get collection

In [ ]:
default_collection = lx.get_collection(collection_name="default")
default_collection

In [ ]:
default_docs = default_collection.list_documents()
default_docs[:5]

### Create a new collection

In [ ]:
from lexy_py.collection.models import Collection

In [ ]:
print(inspect.signature(Collection))

In [ ]:
collection_without_a_client = Collection(collection_name="notbeingcreated")
collection_without_a_client

In [ ]:
with pytest.raises(ValueError) as exc_info:
    collection_without_a_client.list_documents()
print(exc_info)
assert isinstance(exc_info.value, ValueError)
assert str(exc_info.value) == "API client has not been set."

In [ ]:
new_collection = lx.create_collection(
    collection_name="junk", description="just testing for now"
)
new_collection

In [ ]:
lx.list_collections()

In [ ]:
new_collection.list_documents()

In [ ]:
lx.delete_collection(collection_name="junk")

In [ ]:
lx.list_collections()

# Documents

### List documents

In [ ]:
code_docs = lx.list_documents(collection_name="code")
code_docs

In [ ]:
code_doc = code_docs[0]
code_doc

In [ ]:
assert isinstance(code_doc.client, LexyClient)

### Get document

In [ ]:
sample_doc = lx.get_document(code_doc.document_id)
sample_doc

In [ ]:
sample_doc.model_dump()

In [ ]:
assert isinstance(sample_doc.client, LexyClient)

### Create new document

In [ ]:
from lexy_py.document.models import Document

In [ ]:
print(inspect.signature(Document))

In [ ]:
Document("this is a new doc")

In [ ]:
new_docs = lx.add_documents([{"content": "This is my shiny new document!"}])
new_doc = new_docs[0]
new_doc

In [ ]:
lx.delete_document(document_id=new_doc.document_id)

### Upload file documents

In [ ]:
junk_files_collection = lx.create_collection(collection_name="junk_files")
junk_files_collection

In [ ]:
img_docs = lx.document.upload_documents(
    files=[
        "../sample_data/images/lexy-dalle.jpeg",
        "../sample_data/images/lexy.png",
    ],
    collection_name="junk_files",
)
img_docs

In [ ]:
img_docs[0].object_url

In [ ]:
img_docs[0].image

In [ ]:
# uploaded image document
img_docs[0].model_dump()

In [ ]:
# upload image documents in batches
more_img_docs = junk_files_collection.upload_documents(
    files=[
        "../sample_data/images/lexy-dalle.jpeg",
        "../sample_data/images/lexy.png",
        "../sample_data/images/lexy-dalle.jpeg",
    ],
    filenames=["junk1.jpeg", "junk2.jpeg", "junk3.jpeg"],
    batch_size=2,
)
more_img_docs

In [ ]:
assert len(more_img_docs) == 3

In [ ]:
# upload additional files
even_more_docs = junk_files_collection.upload_documents(
    [
        "../sample_data/documents/StarCoder.pdf",
        "../sample_data/documents/fluid.mp4",
        "../sample_data/documents/hotd.txt",
    ]
)
even_more_docs

In [ ]:
assert len(even_more_docs) == 3

In [ ]:
# uploaded PDF document
even_more_docs[0].model_dump()

In [ ]:
# uploaded video document
even_more_docs[1].model_dump()

In [ ]:
# uploaded text document
even_more_docs[2].model_dump()

In [ ]:
lx.delete_collection(collection_name="junk_files", delete_documents=True)

# Indexes

### List indexes

In [ ]:
lx.list_indexes()

### Get index

In [ ]:
idx = lx.get_index("default_text_embeddings")
idx

In [ ]:
docs_for_index = lx.add_documents(
    [
        {"content": "This is one of my docs!"},
        {"content": "This is another one of my docs!"},
    ]
)
docs_for_index

### Query index

In [ ]:
idx.query("hello world")

In [ ]:
results = idx.query("hello world", return_document=True)
result_doc = results[0].get("document")
assert isinstance(result_doc, Document)
assert result_doc.document_id == results[0].get("document_id")

### Create new index

In [ ]:
from lexy_py.index.models import Index

In [ ]:
print(inspect.signature(Index))

# Bindings

### List bindings

In [ ]:
lx.list_bindings()

### Get binding

In [ ]:
binding = lx.get_binding(1)
binding

In [ ]:
binding.collection

In [ ]:
binding.collection.description

In [ ]:
binding.transformer

In [ ]:
binding.index

In [ ]:
binding.index.description

In [ ]:
binding.index.query("hi")

In [ ]:
binding.collection.list_documents(limit=5)

In [ ]:
assert isinstance(binding.collection.client, LexyClient)

### Create new binding

In [ ]:
from lexy_py.binding.models import BindingCreate

In [ ]:
print(inspect.signature(BindingCreate))

## Cleanup

In [ ]:
for d in docs_for_index:
    lx.delete_document(document_id=d.document_id)